In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam




In [10]:
#directory = "/Users/abhar/Downloads/face_age/face_age_image" 

directory = "/Users/abhar/Downloads/practice_imageset" #practice data set

dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels="inferred",
        label_mode="int",
        class_names=None,
        color_mode="rgb",
        batch_size=16,
        image_size=(200, 200),

)

train_x_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.2,subset="training",seed = 42)
validation_x_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.2,subset="validation",seed = 42)



#testx_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.3,subset="validation",seed = 42)
#testy_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.3,subset="validation",seed = 42)
#print(dataset)

#for i in dataset:
    #print(i)

Found 2131 files belonging to 4 classes.
Found 2131 files belonging to 4 classes.
Using 1705 files for training.
Found 2131 files belonging to 4 classes.
Using 426 files for validation.


In [9]:
#Make model
model = Sequential()
model.add(layers.InputLayer(input_shape=(256, 256, 3)))

# 1st conv block
model.add(layers.Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(layers.Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(layers.BatchNormalization())
# 3rd conv block
model.add(layers.Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(layers.BatchNormalization())
# ANN block
model.add(layers.Flatten())
model.add(layers.Dense(units=100, activation='relu'))
model.add(layers.Dense(units=100, activation='relu'))
model.add(layers.Dropout(0.25))
# output layer
model.add(layers.Dense(units=10, activation='softmax'))
model.summary()

opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
im_model = model.fit(x=train_x_ds,epochs=10,validation_data=#should be the y test data here,batch_size=8)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 25)      1900      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 25)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 50)        31300     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 50)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 50)        200       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 70)        31570     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 70)          0

ValueError: `y` argument is not supported when using dataset as input.

In [ ]:
))

#model.fit(x=train_ds, 
    #validation_data=(testImagesX, testY),
    #epochs=200, batch_size=8)


#model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
#history = model.fit(x=train_ds,epochs=10,validation_data=val_ds)